In [56]:
import market
import metrics

import pandas
import numpy
import itertools
import statsmodels.api as sm

https://earlyretirementnow.com/2017/05/24/the-ultimate-guide-to-safe-withdrawal-rates-part-15-sequence-of-return-risk-part2/

In [57]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [58]:
m = market.Returns_US_1871()
years = 30
first_year = m.start_year
last_year = 2018 - years + 1

swrs = pandas.DataFrame(index=numpy.arange(first_year, last_year+1), columns=['SWR'])
for start in range(first_year, last_year + 1):
    returns = []
    
    for one_year in zip(range(years), m.iter_from(start)):
        annual_returns = one_year[1]
        real_returns = ((1 + annual_returns.stocks) / (1 + annual_returns.inflation)) - 1
        returns.append(real_returns)        

    swr = float(metrics.pwa(1, 1, returns))
    swrs.loc[start] = swr
swrs.head()

,SWR
1871,0.0803345
1872,0.0772254
1873,0.0762225
1874,0.079344
1875,0.0781118


In [191]:
def get_averages(slice_by=5):
    m = market.Returns_US_1871()
    years = 30
    first_year = m.start_year
    last_year = 2018 - years + 1
    num_slices = years // slice_by
    
    return_columns = ['Return%s' % i for i in range(num_slices)]
    inflation_columns = ['Inflation%s' % i for i in range(num_slices)]

    series = pandas.DataFrame(index=numpy.arange(first_year, last_year+1), columns=return_columns + inflation_columns)
    for start in range(first_year, last_year + 1):
        annual = zip(range(years), m.iter_from(start))
        annual = [x[1] for x in annual]

        returns = [x.stocks for x in annual]
        inflation = [x.inflation for x in annual]

        average_inflation = [metrics.average(x) for x in grouper(inflation, slice_by)]
        average_return = [metrics.average(x) for x in grouper(returns, slice_by)]
        row = series.loc[start]
        
        for i in range(num_slices):
            row['Return%s' % i] = average_return[i]
            row['Inflation%s' %i] = average_inflation[i]
    return series
        
averages = get_averages(slice_by=2)
averages.head()

,Return0,Return1,Return2,Return3,Return4,Return5,Return6,Return7,Return8,Return9,...,Inflation5,Inflation6,Inflation7,Inflation8,Inflation9,Inflation10,Inflation11,Inflation12,Inflation13,Inflation14
1871,0.131351,0.00917582,-0.0456835,0.0612705,0.370543,0.020143,-0.0880874,0.197286,0.0116552,0.00300656,...,0.03106,-0.089643,-0.017242,0.00108242,-0.0113095,0.00847348,-0.0870983,-0.00703961,0.0218489,0.0724585
1872,0.0431736,0.0450891,-0.0853421,0.317973,0.133341,-0.0096656,0.0789723,0.0542135,0.0477407,0.0605056,...,-0.0474426,-0.0687861,0.0238114,-0.0465384,-0.017988,-0.027304,-0.0135869,0.000420163,0.0916502,0.000297508
1873,0.00917582,-0.0456835,0.0612705,0.370543,0.020143,-0.0880874,0.197286,0.0116552,0.00300656,0.121002,...,-0.089643,-0.017242,0.00108242,-0.0113095,0.00847348,-0.0870983,-0.00703961,0.0218489,0.0724585,0.0605391
1874,0.0450891,-0.0853421,0.317973,0.133341,-0.0096656,0.0789723,0.0542135,0.0477407,0.0605056,-0.0633373,...,-0.0687861,0.0238114,-0.0465384,-0.017988,-0.027304,-0.0135869,0.000420163,0.0916502,0.000297508,0.0262151
1875,-0.0456835,0.0612705,0.370543,0.020143,-0.0880874,0.197286,0.0116552,0.00300656,0.121002,-0.0779632,...,-0.017242,0.00108242,-0.0113095,0.00847348,-0.0870983,-0.00703961,0.0218489,0.0724585,0.0605391,-0.0104861


In [215]:
Y = swrs.astype(float)
#return_columns = ['Return%s' % i for i in range(4)]
inflation_columns = ['Inflation%s' % i for i in range(14)]
X = averages[inflation_columns].astype(float)
#X = averages.astype(float)
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    SWR   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.214
Method:                 Least Squares   F-statistic:                     5.519
Date:                Fri, 12 Apr 2019   Prob (F-statistic):           8.08e-08
Time:                        19:21:55   Log-Likelihood:                 307.47
No. Observations:                 119   AIC:                            -584.9
Df Residuals:                     104   BIC:                            -543.3
Df Model:                          14                                         
Covariance Type:                  HAC                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0661      0.005     13.591      0.000       0.057       0.076
Inflation0      0.0332      0.049      0.679      0.497      -0.063       0.129
Inflation1      0.0308      0.064      0.479      0.632      -0.095       0.157
Inflation2     -0.0210      0.047     -0.443      0.658      -0.114       0.072
Inflation3     -0.1088      0.042     -2.588      0.010      -0.191      -0.026
Inflation4     -0.0325      0.037     -0.871      0.384      -0.105       0.041
Inflation5     -0.1041      0.035     -2.952      0.003      -0.173      -0.035
Inflation6     -0.0495      0.035     -1.424      0.154      -0.118       0.019
Inflation7     -0.0123      0.049     -0.250      0.803      -0.109       0.084
Inflation8     -0.0534      0.034     -1.553      0.120      -0.121       0.014
Inflation9     -0.0212      0.033     -0.634      0.526      -0.087       0.044
Inflation10     0.0190      0.037      0.516      0.606      -0.053       0.091
Inflation11     0.0164      0.039      0.426      0.670      -0.059       0.092
Inflation12     0.0492      0.056      0.875      0.381      -0.061       0.159
Inflation13     0.0309      0.058      0.532      0.595      -0.083       0.145
==============================================================================
Omnibus:                        0.709   Durbin-Watson:                   0.385
Prob(Omnibus):                  0.702   Jarque-Bera (JB):                0.812
Skew:                           0.170   Prob(JB):                        0.666
Kurtosis:                       2.781   Cond. No.                         39.0
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 5 lags and without small sample correction
"""

In [120]:
model.params[["Inflation0", "Inflation1", "Inflation2" "Inflation3", "Inflation4", "Inflation5"]].sum()

-0.5504788366825107